In [1]:
from __future__ import print_function
import tensorflow as tf
import keras
import pickle
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model 
from models import resnext, resnet_v1, resnet_v2, mobilenets, inception_v3, inception_resnet_v2, densenet
from utils import lr_schedule
import numpy as np
import os
from sklearn.model_selection import train_test_split

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

Using TensorFlow backend.


In [2]:
# Training parameters
batch_size = 16
epochs = 50
data_augmentation = False
num_classes = 1
subtract_pixel_mean = True  # Subtracting pixel mean improves accuracy
base_model = 'resnet20'
# Choose what attention_module to use: cbam_block / se_block / None
attention_module = 'cbam_block'
model_type = base_model if attention_module==None else base_model+'_'+attention_module

In [3]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [4]:
X = np.array(X)
y = np.array(y)

In [5]:
X.shape

(9837, 256, 256, 3)

In [6]:
y.shape

(9837,)

In [7]:
input_shape = X.shape[1:]

In [8]:
print(X.shape[1:])

(256, 256, 3)


In [10]:
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  shuffle=True, stratify=y, 
                                                  test_size=0.2, random_state=42)

In [11]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')
print('y_train shape:', y_train.shape)

x_train shape: (7869, 256, 256, 3)
7869 train samples
1968 validation samples
y_train shape: (7869,)


In [12]:
#depth = 20 # For ResNet, specify the depth (e.g. ResNet50: depth=50)
#model = resnet_v1.resnet_v1(input_shape=input_shape, depth=depth, attention_module=attention_module)
# model = resnet_v2.resnet_v2(input_shape=input_shape, depth=depth, attention_module=attention_module)   
# model = resnext.ResNext(input_shape=input_shape, classes=num_classes, attention_module=attention_module)
# model = mobilenets.MobileNet(input_shape=input_shape, classes=num_classes, attention_module=attention_module)
#model = inception_v3.InceptionV3(include_top=True, input_shape=input_shape, classes=1, attention_module=attention_module)
# model = inception_resnet_v2.InceptionResNetV2(input_shape=input_shape, classes=num_classes, attention_module=attention_module)
# model = densenet.DenseNet(input_shape=input_shape, classes=num_classes, attention_module=attention_module)

In [13]:
model = inception_v3.InceptionV3(include_top=True, input_shape=input_shape, 
                                 classes=num_classes, classifier_activation="sigmoid",
                                 attention_module=attention_module)

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)), # (lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Learning rate:  0.0001
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
________________________________________________________________

In [15]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = save_dir + '/' + 'best_model_sigmoid.h5'

In [16]:


# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler] # [checkpoint, lr_reducer, lr_scheduler]

In [17]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
    
    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_val, y_val),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

Not using data augmentation.
Learning rate:  0.0001
Epoch 1/50
492/492 [==============================] - ETA: 0s - loss: 0.7028 - accuracy: 0.5085 ETA: 4s - los - ETA: 1s - loss: 0.7029 - WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0267s vs `on_test_batch_end` time: 0.0423s). Check your callbacks.

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to /home/trojan/Desktop/dimentia/attention/CBAM/Implementation/CBAM-keras-master/saved_models/best_model_sigmoid.h5
492/492 [==============================] - 111s 226ms/step - loss: 0.7028 - accuracy: 0.5085 - val_loss: 0.7579 - val_accuracy: 0.5000
Learning rate:  0.0001
Epoch 2/50
492/492 [==============================] - ETA: 0s - loss: 0.6903 - accuracy: 0.5392
Epoch 00002: val_accuracy improved from 0.50000 to 0.56657, saving model to /home/trojan/Desktop/dimentia/attention/CBAM/Implementation/CBAM-keras-master/saved_models/best_model_sigmoid.h5
492/492 

Learning rate:  0.0001
Epoch 9/50
492/492 [==============================] - ETA: 0s - loss: 0.4171 - accuracy: 0.8093
Epoch 00009: val_accuracy improved from 0.71189 to 0.75203, saving model to /home/trojan/Desktop/dimentia/attention/CBAM/Implementation/CBAM-keras-master/saved_models/best_model_sigmoid.h5
492/492 [==============================] - 104s 212ms/step - loss: 0.4171 - accuracy: 0.8093 - val_loss: 0.5726 - val_accuracy: 0.7520
Learning rate:  0.0001
Epoch 10/50
492/492 [==============================] - ETA: 0s - loss: 0.3315 - accuracy: 0.8584
Epoch 00010: val_accuracy did not improve from 0.75203
492/492 [==============================] - 103s 210ms/step - loss: 0.3315 - accuracy: 0.8584 - val_loss: 0.8861 - val_accuracy: 0.6926
Learning rate:  0.0001
Epoch 11/50
492/492 [==============================] - ETA: 0s - loss: 0.2734 - accuracy: 0.8865
Epoch 00011: val_accuracy improved from 0.75203 to 0.79726, saving model to /home/trojan/Desktop/dimentia/attention/CBAM/Implem

492/492 [==============================] - ETA: 0s - loss: 9.0835e-04 - accuracy: 0.9997
Epoch 00049: val_accuracy did not improve from 0.96850
492/492 [==============================] - 103s 210ms/step - loss: 9.0835e-04 - accuracy: 0.9997 - val_loss: 0.1369 - val_accuracy: 0.9649
Learning rate:  1e-05
Epoch 50/50
492/492 [==============================] - ETA: 0s - loss: 3.0247e-04 - accuracy: 1.0000
Epoch 00050: val_accuracy improved from 0.96850 to 0.97002, saving model to /home/trojan/Desktop/dimentia/attention/CBAM/Implementation/CBAM-keras-master/saved_models/best_model_sigmoid.h5
492/492 [==============================] - 104s 212ms/step - loss: 3.0247e-04 - accuracy: 1.0000 - val_loss: 0.1320 - val_accuracy: 0.9700


In [18]:
model = load_model(filepath)

In [19]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [20]:
# Score trained model.
scores = model.evaluate(x_val, y_val, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

62/62 [==============================] - 7s 113ms/step - loss: 0.1320 - accuracy: 0.9700
Test loss: 0.13198120892047882
Test accuracy: 0.9700203537940979
